In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
size(d) # Should be 504x8

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


(504, 8)

pulled_left, condition, prosoc_left

In [2]:
@model m10_3(y, x₁, x₂) = begin
    α ~ Normal(0, 10)
    βp ~ Normal(0, 10)
    βpC ~ Normal(0, 10)

    for i ∈ 1:length(y)
        p = logistic(α + (βp + βpC * x₁[i]) * x₂[i])
        y[i] ~ Binomial(1, p)
    end
end;

posterior = sample(m10_3(d[:,:pulled_left], d[:,:condition], d[:,:prosoc_left]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.2
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.05751073520096119, std = [1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 348.9359764449993;
  #lf / sample        = 0.0;
  #evals / sample     = 31.686;
  pre-cond. metric    = [1.0, 1.0, 1.0].


Fix the inclusion of adaptation samples

In [3]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

Rethinking result

In [4]:
m_10_03t_result = "
      Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
 a    0.05   0.13      -0.15       0.25  3284    1
 bp   0.62   0.22       0.28       0.98  3032    1
 bpC -0.11   0.26      -0.53       0.29  3184    1
";

Describe the draws

In [5]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean           SD        Naive SE               MCSE                ESS   
       α    0.047644387  0.130079828 0.0041134853 0.0056361364249392191527743  532.66838
     βpC   -0.098425696  0.274451289 0.0086789118 0.0127949725573689807728295  460.09958
  lf_num    0.000000000  0.000000000 0.0000000000 0.0000000000000000000000000        NaN
 elapsed    0.182290972  0.114010241 0.0036053204 0.0068662591221784431347319  275.70672
 epsilon    0.057510735  0.000000000 0.0000000000 0.0000000000000000023129646    0.00000
      βp    0.609956880  0.246700756 0.0078013629 0.0140384031043162132990076  308.82009
      lp -349.387004281  1.272188914 0.0402301458 0.0696864997378833644914309  333.27744
eval_num   32.452000000 15.181112020 0.4800689140 0.3529079200018045425757407 1000.00000
  lf_eps    0.057510735  0.000000000 0.0000000000 0.0000000000000000023129646    0.00000


End of m_10_03t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*